In [1]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import scale
from make_scores import make_scores
from regression_table_significance import regression_table
import scipy.stats as stats
import numpy as np
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv("..\..\data\mturk_experiment_2.csv",encoding="unicode escape")
feature_names = ["Informativeness","Implicature","Length in Words","Length in Characters","F-score","I-score","Lexical Density"]
features = data[feature_names]
target = data["Formality"]
cv = KFold(n_splits=10, random_state=1, shuffle=True)
scoring = ["r2","neg_mean_squared_error","neg_mean_absolute_error","max_error"]

In [3]:
# Create datasets

#SQUINKY!
squinky = pd.read_csv("..\..\data\mturk_experiment_2.csv",encoding="unicode escape")
lexical = pd.read_csv("..\..\data\squinky_lexical.csv",encoding="unicode escape")
squinky_lexical = pd.merge(squinky,lexical)
feature_names = ["Informativeness","Implicature","Length in Words","Length in Characters","F-score","I-score","Lexical Density"]
features = squinky_lexical[feature_names]
target = squinky_lexical["Formality"]
squinky_x, squinky_test_x, squinky_y, squinky_test_y = train_test_split(features, target, test_size=0.2, random_state=5)


# Lexical
squinky = pd.read_csv("..\..\data\mturk_experiment_2.csv",encoding="unicode escape")
lexical = pd.read_csv("..\..\data\squinky_lexical.csv",encoding="unicode escape")
squinky_lexical = pd.merge(squinky,lexical)
print(len(squinky_lexical))
feature_names = ["Informativeness","Implicature","Length in Words","Length in Characters","F-score","I-score","Lexical Density","Adjective Density","Spellcheck Percentage","Syllable Ratio","Latinate vs Germanic"]
features = squinky_lexical[feature_names]
target = squinky_lexical["Formality"]
lexical_x, lexical_test_x, lexical_y, lexical_test_y = train_test_split(features, target, test_size=0.2, random_state=5)

# Scores
squinky = pd.read_csv("..\..\data\mturk_experiment_2.csv",encoding="unicode escape")
scores = pd.read_csv("..\..\data\squinky_scores.csv",encoding="unicode escape")
squinky_scores = pd.merge(squinky,scores)
feature_names = ["Informativeness","Implicature","Length in Words","Length in Characters","F-score","I-score","Lexical Density","ADF Score","FK Grade Level","Reading Time","WF Score"]
features = squinky_scores[feature_names]
target = squinky_scores["Formality"]
scores_x, scores_test_x, scores_y, scores_test_y = train_test_split(features, target, test_size=0.2, random_state=5)

# Reading
squinky = pd.read_csv("..\..\data\mturk_experiment_2.csv",encoding="unicode escape")
reading = pd.read_csv("../../data/squinky_reading.csv",encoding="unicode escape")
squinky_reading = pd.merge(squinky,reading)
feature_names = ["Informativeness","Implicature","Length in Words","Length in Characters","F-score","I-score","Lexical Density","FK Reading Ease","FOG Scale","SMOG Index","ARI","CL Index","LW Formula","DC Score","Readability Consensus","Spache Formula"]
features = squinky_reading[feature_names]
target = squinky_reading["Formality"]
reading_x, reading_test_x, reading_y, reading_test_y = train_test_split(features, target, test_size=0.2, random_state=5)

# NLP
squinky = pd.read_csv("..\..\data\mturk_experiment_2.csv",encoding="unicode escape")
nlp = pd.read_csv("..\..\data\\squinky_nlp.csv",encoding="unicode escape")
squinky_nlp = pd.merge(squinky,nlp)
feature_names = ["Informativeness","Implicature","Length in Words","Length in Characters","F-score","I-score","Lexical Density","Cluster","Topic"]
features = squinky_nlp[feature_names]
target = squinky_nlp["Formality"]
nlp_x, nlp_test_x, nlp_y, nlp_test_y = train_test_split(features, target, test_size=0.2, random_state=5)

7000


In [4]:
# Linear regression
linear_regression = pd.DataFrame()

# SQUINKY!
lin_reg_squinky = list(linear_model.LinearRegression().fit(squinky_x, squinky_y).predict(squinky_test_x))
df = pd.DataFrame()
df["True"] = list(squinky_test_y)
df["Predicted"] = lin_reg_squinky
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])

linear_regression["Squinky"] = squared_error

# Lexical
lin_reg_lexical = list(linear_model.LinearRegression().fit(lexical_x, lexical_y).predict(lexical_test_x))
df_lex = pd.DataFrame()
df_lex["True"] = list(lexical_test_y)
df_lex["Predicted"] = lin_reg_lexical
squared_error = []
for index in df_lex.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Lexical"] = squared_error

# Scores
lin_reg_scores = list(linear_model.LinearRegression().fit(scores_x, scores_y).predict(scores_test_x))
df_score = pd.DataFrame()
df_score["True"] = list(scores_test_y)
df_score["Predicted"] = lin_reg_scores
squared_error = []
for index in df_score.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Scores"] = squared_error

# Reading
lin_reg_reading = list(linear_model.LinearRegression().fit(reading_x, reading_y).predict(reading_test_x))
df = pd.DataFrame()
df["True"] = list(reading_test_y)
df["Predicted"] = lin_reg_reading
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Reading"] = squared_error

# NLP
lin_reg_nlp = list(linear_model.LinearRegression().fit(nlp_x, nlp_y).predict(nlp_test_x))
df = pd.DataFrame()
df["True"] = list(nlp_test_y)
df["Predicted"] = lin_reg_nlp
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["NLP"] = squared_error

linear_regression

linear_regression_significances = pd.DataFrame()
t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Lexical"])
linear_regression_significances["Lexical - Linear Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Scores"])
linear_regression_significances["Scores - Linear Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Reading"])
linear_regression_significances["Reading - Linear Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["NLP"])
linear_regression_significances["NLP - Linear Regression"] = [p,p<0.05]

linear_regression_significances.T

,0,1
Lexical - Linear Regression,1.0,False
Scores - Linear Regression,1.0,False
Reading - Linear Regression,0.958639,False
NLP - Linear Regression,0.969001,False


In [5]:
# Linear regression
linear_regression = pd.DataFrame()

# SQUINKY!
lin_reg_squinky = list(linear_model.Ridge().fit(squinky_x, squinky_y).predict(squinky_test_x))
df = pd.DataFrame()
df["True"] = list(squinky_test_y)
df["Predicted"] = lin_reg_squinky
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])

linear_regression["Squinky"] = squared_error

# Lexical
lin_reg_lexical = list(linear_model.Ridge().fit(lexical_x, lexical_y).predict(lexical_test_x))
df_lex = pd.DataFrame()
df_lex["True"] = list(lexical_test_y)
df_lex["Predicted"] = lin_reg_lexical
squared_error = []
for index in df_lex.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Lexical"] = squared_error

# Scores
lin_reg_scores = list(linear_model.Ridge().fit(scores_x, scores_y).predict(scores_test_x))
df_score = pd.DataFrame()
df_score["True"] = list(scores_test_y)
df_score["Predicted"] = lin_reg_scores
squared_error = []
for index in df_score.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Scores"] = squared_error

# Reading
lin_reg_reading = list(linear_model.Ridge().fit(reading_x, reading_y).predict(reading_test_x))
df = pd.DataFrame()
df["True"] = list(reading_test_y)
df["Predicted"] = lin_reg_reading
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Reading"] = squared_error

# NLP
lin_reg_nlp = list(linear_model.Ridge().fit(nlp_x, nlp_y).predict(nlp_test_x))
df = pd.DataFrame()
df["True"] = list(nlp_test_y)
df["Predicted"] = lin_reg_nlp
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["NLP"] = squared_error

linear_regression

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Lexical"])
linear_regression_significances["Lexical - Ridge Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Scores"])
linear_regression_significances["Scores - Ridge Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Reading"])
linear_regression_significances["Reading - Ridge Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["NLP"])
linear_regression_significances["NLP - Ridge Regression"] = [p,p<0.05]

linear_regression_significances.T

,0,1
Lexical - Linear Regression,1.0,False
Scores - Linear Regression,1.0,False
Reading - Linear Regression,0.958639,False
NLP - Linear Regression,0.969001,False
Lexical - Ridge Regression,1.0,False
Scores - Ridge Regression,1.0,False
Reading - Ridge Regression,0.958623,False
NLP - Ridge Regression,0.968995,False


In [6]:
# Linear regression
linear_regression = pd.DataFrame()

# SQUINKY!
lin_reg_squinky = list(linear_model.Lasso().fit(squinky_x, squinky_y).predict(squinky_test_x))
df = pd.DataFrame()
df["True"] = list(squinky_test_y)
df["Predicted"] = lin_reg_squinky
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])

linear_regression["Squinky"] = squared_error

# Lexical
lin_reg_lexical = list(linear_model.Lasso().fit(lexical_x, lexical_y).predict(lexical_test_x))
df_lex = pd.DataFrame()
df_lex["True"] = list(lexical_test_y)
df_lex["Predicted"] = lin_reg_lexical
squared_error = []
for index in df_lex.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Lexical"] = squared_error

# Scores
lin_reg_scores = list(linear_model.Lasso().fit(scores_x, scores_y).predict(scores_test_x))
df_score = pd.DataFrame()
df_score["True"] = list(scores_test_y)
df_score["Predicted"] = lin_reg_scores
squared_error = []
for index in df_score.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Scores"] = squared_error

# Reading
lin_reg_reading = list(linear_model.Lasso().fit(reading_x, reading_y).predict(reading_test_x))
df = pd.DataFrame()
df["True"] = list(reading_test_y)
df["Predicted"] = lin_reg_reading
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Reading"] = squared_error

# NLP
lin_reg_nlp = list(linear_model.Lasso().fit(nlp_x, nlp_y).predict(nlp_test_x))
df = pd.DataFrame()
df["True"] = list(nlp_test_y)
df["Predicted"] = lin_reg_nlp
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["NLP"] = squared_error

linear_regression

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Lexical"])
linear_regression_significances["Lexical - Lasso Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Scores"])
linear_regression_significances["Scores - Lasso Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Reading"])
linear_regression_significances["Reading - Lasso Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["NLP"])
linear_regression_significances["NLP - Lasso Regression"] = [p,p<0.05]

linear_regression_significances.T

,0,1
Lexical - Linear Regression,1.0,False
Scores - Linear Regression,1.0,False
Reading - Linear Regression,0.958639,False
NLP - Linear Regression,0.969001,False
Lexical - Ridge Regression,1.0,False
Scores - Ridge Regression,1.0,False
Reading - Ridge Regression,0.958623,False
NLP - Ridge Regression,0.968995,False
Lexical - Lasso Regression,1.0,False
Scores - Lasso Regression,1.0,False


In [7]:
# Linear regression
linear_regression = pd.DataFrame()

# SQUINKY!
lin_reg_squinky = list(linear_model.Lasso().fit(squinky_x, squinky_y).predict(squinky_test_x))
df = pd.DataFrame()
df["True"] = list(squinky_test_y)
df["Predicted"] = lin_reg_squinky
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])

linear_regression["Squinky"] = squared_error

# Lexical
lin_reg_lexical = list(linear_model.Lasso().fit(lexical_x, lexical_y).predict(lexical_test_x))
df_lex = pd.DataFrame()
df_lex["True"] = list(lexical_test_y)
df_lex["Predicted"] = lin_reg_lexical
squared_error = []
for index in df_lex.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Lexical"] = squared_error

# Scores
lin_reg_scores = list(linear_model.Lasso().fit(scores_x, scores_y).predict(scores_test_x))
df_score = pd.DataFrame()
df_score["True"] = list(scores_test_y)
df_score["Predicted"] = lin_reg_scores
squared_error = []
for index in df_score.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Scores"] = squared_error

# Reading
lin_reg_reading = list(linear_model.Lasso().fit(reading_x, reading_y).predict(reading_test_x))
df = pd.DataFrame()
df["True"] = list(reading_test_y)
df["Predicted"] = lin_reg_reading
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Reading"] = squared_error

# NLP
lin_reg_nlp = list(linear_model.Lasso().fit(nlp_x, nlp_y).predict(nlp_test_x))
df = pd.DataFrame()
df["True"] = list(nlp_test_y)
df["Predicted"] = lin_reg_nlp
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["NLP"] = squared_error

linear_regression

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Lexical"])
linear_regression_significances["Lexical - Lasso Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Scores"])
linear_regression_significances["Scores - Lasso Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Reading"])
linear_regression_significances["Reading - Lasso Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["NLP"])
linear_regression_significances["NLP - Lasso Regression"] = [p,p<0.05]

linear_regression_significances.T

,0,1
Lexical - Linear Regression,1.0,False
Scores - Linear Regression,1.0,False
Reading - Linear Regression,0.958639,False
NLP - Linear Regression,0.969001,False
Lexical - Ridge Regression,1.0,False
Scores - Ridge Regression,1.0,False
Reading - Ridge Regression,0.958623,False
NLP - Ridge Regression,0.968995,False
Lexical - Lasso Regression,1.0,False
Scores - Lasso Regression,1.0,False


In [8]:
# Linear regression
linear_regression = pd.DataFrame()

# SQUINKY!
lin_reg_squinky = list(linear_model.LassoLars().fit(squinky_x, squinky_y).predict(squinky_test_x))
df = pd.DataFrame()
df["True"] = list(squinky_test_y)
df["Predicted"] = lin_reg_squinky
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])

linear_regression["Squinky"] = squared_error

# Lexical
lin_reg_lexical = list(linear_model.LassoLars().fit(lexical_x, lexical_y).predict(lexical_test_x))
df_lex = pd.DataFrame()
df_lex["True"] = list(lexical_test_y)
df_lex["Predicted"] = lin_reg_lexical
squared_error = []
for index in df_lex.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Lexical"] = squared_error

# Scores
lin_reg_scores = list(linear_model.LassoLars().fit(scores_x, scores_y).predict(scores_test_x))
df_score = pd.DataFrame()
df_score["True"] = list(scores_test_y)
df_score["Predicted"] = lin_reg_scores
squared_error = []
for index in df_score.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Scores"] = squared_error

# Reading
lin_reg_reading = list(linear_model.LassoLars().fit(reading_x, reading_y).predict(reading_test_x))
df = pd.DataFrame()
df["True"] = list(reading_test_y)
df["Predicted"] = lin_reg_reading
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Reading"] = squared_error

# NLP
lin_reg_nlp = list(linear_model.LassoLars().fit(nlp_x, nlp_y).predict(nlp_test_x))
df = pd.DataFrame()
df["True"] = list(nlp_test_y)
df["Predicted"] = lin_reg_nlp
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["Predicted"][index])
linear_regression["NLP"] = squared_error

linear_regression

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Lexical"])
linear_regression_significances["Lexical - LARS Lasso Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Scores"])
linear_regression_significances["Scores - LARS Lasso Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Reading"])
linear_regression_significances["Reading - LARS Lasso Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["NLP"])
linear_regression_significances["NLP - LARS Lasso Regression"] = [p,p<0.05]

linear_regression_significances.T

C:\Users\jack-\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLars())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
C:\Users\jack-\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_base.py:133:

,0,1
Lexical - Linear Regression,1.0,False
Scores - Linear Regression,1.0,False
Reading - Linear Regression,0.958639,False
NLP - Linear Regression,0.969001,False
Lexical - Ridge Regression,1.0,False
Scores - Ridge Regression,1.0,False
Reading - Ridge Regression,0.958623,False
NLP - Ridge Regression,0.968995,False
Lexical - Lasso Regression,1.0,False
Scores - Lasso Regression,1.0,False


In [9]:
# Linear regression
linear_regression = pd.DataFrame()

# SQUINKY!
lin_reg_squinky = list(linear_model.BayesianRidge().fit(squinky_x, squinky_y).predict(squinky_test_x))
df = pd.DataFrame()
df["True"] = list(squinky_test_y)
df["Predicted"] = lin_reg_squinky
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])

linear_regression["Squinky"] = squared_error

# Lexical
lin_reg_lexical = list(linear_model.BayesianRidge().fit(lexical_x, lexical_y).predict(lexical_test_x))
df_lex = pd.DataFrame()
df_lex["True"] = list(lexical_test_y)
df_lex["Predicted"] = lin_reg_lexical
squared_error = []
for index in df_lex.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Lexical"] = squared_error

# Scores
lin_reg_scores = list(linear_model.BayesianRidge().fit(scores_x, scores_y).predict(scores_test_x))
df_score = pd.DataFrame()
df_score["True"] = list(scores_test_y)
df_score["Predicted"] = lin_reg_scores
squared_error = []
for index in df_score.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Scores"] = squared_error

# Reading
lin_reg_reading = list(linear_model.BayesianRidge().fit(reading_x, reading_y).predict(reading_test_x))
df = pd.DataFrame()
df["True"] = list(reading_test_y)
df["Predicted"] = lin_reg_reading
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Reading"] = squared_error

# NLP
lin_reg_nlp = list(linear_model.BayesianRidge().fit(nlp_x, nlp_y).predict(nlp_test_x))
df = pd.DataFrame()
df["True"] = list(nlp_test_y)
df["Predicted"] = lin_reg_nlp
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["Predicted"][index])
linear_regression["NLP"] = squared_error

linear_regression

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Lexical"])
linear_regression_significances["Lexical - Bayesian Ridge Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Scores"])
linear_regression_significances["Scores - Bayesian Ridge Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Reading"])
linear_regression_significances["Reading - Bayesian Ridge Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["NLP"])
linear_regression_significances["NLP - Bayesian Ridge Regression"] = [p,p<0.05]

linear_regression_significances.T

,0,1
Lexical - Linear Regression,1.0,False
Scores - Linear Regression,1.0,False
Reading - Linear Regression,0.958639,False
NLP - Linear Regression,0.969001,False
Lexical - Ridge Regression,1.0,False
Scores - Ridge Regression,1.0,False
Reading - Ridge Regression,0.958623,False
NLP - Ridge Regression,0.968995,False
Lexical - Lasso Regression,1.0,False
Scores - Lasso Regression,1.0,False


In [10]:
# Linear regression
linear_regression = pd.DataFrame()

# SQUINKY!
lin_reg_squinky = list(linear_model.SGDRegressor().fit(squinky_x, squinky_y).predict(squinky_test_x))
df = pd.DataFrame()
df["True"] = list(squinky_test_y)
df["Predicted"] = lin_reg_squinky
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])

linear_regression["Squinky"] = squared_error

# Lexical
lin_reg_lexical = list(linear_model.SGDRegressor().fit(lexical_x, lexical_y).predict(lexical_test_x))
df_lex = pd.DataFrame()
df_lex["True"] = list(lexical_test_y)
df_lex["Predicted"] = lin_reg_lexical
squared_error = []
for index in df_lex.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Lexical"] = squared_error

# Scores
lin_reg_scores = list(linear_model.SGDRegressor().fit(scores_x, scores_y).predict(scores_test_x))
df_score = pd.DataFrame()
df_score["True"] = list(scores_test_y)
df_score["Predicted"] = lin_reg_scores
squared_error = []
for index in df_score.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Scores"] = squared_error

# Reading
lin_reg_reading = list(linear_model.SGDRegressor().fit(reading_x, reading_y).predict(reading_test_x))
df = pd.DataFrame()
df["True"] = list(reading_test_y)
df["Predicted"] = lin_reg_reading
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["True"][index])
linear_regression["Reading"] = squared_error

# NLP
lin_reg_nlp = list(linear_model.SGDRegressor().fit(nlp_x, nlp_y).predict(nlp_test_x))
df = pd.DataFrame()
df["True"] = list(nlp_test_y)
df["Predicted"] = lin_reg_nlp
squared_error = []
for index in df.index:
    squared_error.append(df["Predicted"][index]-df["Predicted"][index])
linear_regression["NLP"] = squared_error

linear_regression

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Lexical"])
linear_regression_significances["Lexical - SGD Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Scores"])
linear_regression_significances["Scores - SGD Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["Reading"])
linear_regression_significances["Reading - SGD Regression"] = [p,p<0.05]

t, p = stats.ttest_ind(linear_regression["Squinky"], linear_regression["NLP"])
linear_regression_significances["NLP - SGD Regression"] = [p,p<0.05]

linear_regression_significances.T

,0,1
Lexical - Linear Regression,1.0,False
Scores - Linear Regression,1.0,False
Reading - Linear Regression,0.958639,False
NLP - Linear Regression,0.969001,False
Lexical - Ridge Regression,1.0,False
Scores - Ridge Regression,1.0,False
Reading - Ridge Regression,0.958623,False
NLP - Ridge Regression,0.968995,False
Lexical - Lasso Regression,1.0,False
Scores - Lasso Regression,1.0,False
